In [1]:
%run 01-model.ipynb

from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield

from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization, NADH_NADPH

'' is not a valid SBML 'SId'.


#### Co-factor swapping targets

In [2]:
model.solver = "glpk"

In [3]:
biomass = model.reactions.DRAIN_Biomass

In [4]:
biomass.lower_bound = 5 # OBS denne skal måske ikke være 0,1 tjek tutorial notebooken og forstå det


In [5]:
target = model.metabolites.get_by_id("psi")

In [6]:
demand = model.reactions.DM_psi

The demand reaction for the "target" (psilocybin) is defined in "01-model.ipynb"

In [7]:
model.objective = demand

In [8]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

In [9]:
CofactorSwapOptimization?

Init signature:
CofactorSwapOptimization(
    cofactor_id_swaps=(['nad_c', 'nadh_c'], ['nadp_c', 'nadph_c']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)
Docstring:     
Optimize co-factor swapping

As suggested in [1]_, flux through a given reaction can sometimes be optimized by swapping complementary
co-factor. This class implements a search for reactions when swapped improve the given objective. Briefly,
the approach is to

- find reactions that have all the targeted co-factor pairs e.g. (nad_c -> nadp_c, nadh_c -> nadph_c)

- add reactions that have the co-factors swapped and then by a search algorithm switching one off in favor of the
  other

The implementation here differs from that in [1]_ in that we use a general purpose search algorithm rather than
formulating the search as a mixed integer linear programming problem.

References
----------
.. [1] King, Zachary A., and Adam M. Feist. "Optimizing Cofactor Specificity of Oxidoreductase Enzym

In [10]:
swap_opt = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']))

In [11]:
%%time
result = swap_opt.run(max_size=5)

Starting optimization at Mon, 23 Nov 2020 11:41:54


HBox()

Finished after 00:00:42
CPU times: user 1min 21s, sys: 0 ns, total: 1min 21s
Wall time: 1min 21s


In [12]:
result

,index,targets,fitness


AND NOW FOR TRP 

In [13]:
target = model.metabolites.get_by_id("TRP")

In [14]:
demand = model.add_boundary(target, type="demand")

In [15]:
model.objective = demand

In [16]:
product_yield = product_yield(demand, model.reactions.BOUNDARY_GLCe)

TypeError: __call__() missing 1 required positional argument: 'targets'

In [ ]:
swap_opt2 = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']))

In [ ]:
%%time
result2 = swap_opt2.run(max_size=5)

In [ ]:
result2

In [ ]:
?cofactor_id_swaps

In [ ]:
?swap_cofactors

In [ ]:
?ReactionCofactorSwapTarget

In [ ]:
?CofactorSwapOptimization

In [ ]:
swap_optimization = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True)

In [ ]:
NADH_NADPH = (['NAD', 'NADH'],['NADP', 'NADPH'])

In [ ]:
print(NADH_NADPH)

In [ ]:
cofactor_id_swaps = NADH_NADPH

In [ ]:
cofactor_id_swaps

In [ ]:
cofactor_id_swaps = ([from 'nad_c'], [to 'NAD'])

In [ ]:
swap_optimization2 = CofactorSwapOptimization(model=model, objective_function=product_yield, plot=True, cofactor_id_swaps)

In [ ]:
CofactorSwapOptimization(
    cofactor_id_swaps=(['NAD', 'NADH'], ['NADP', 'NADPH']),
    candidate_reactions=None,
    skip_reactions=None,
    *args,
    **kwargs,
)

Do we even have any cofactors?

PsiD: pyridoxal-5′-phosphate (PLP) i planter, men hvad med svampe?

PSIH: er en cytochrome P450 (so guess it use heme as cofactor?)
PcCpr: er en cp450 reductase som har cofaktorerne (FMN, FAD and NADPH)


PsiK: ATP 

PsiM x2: S-adenosyl-L-methionine (SAM) 

Skal ikke tilføje cofaktorerne i reaktionenerne! 


situationen med 2 enzymer i reaktion 2 forklaret:
så rekation 2 består altså både af  cytochrome P450 gene PcpsiH OG en cytochrome P450 reductase (CPR) partner 

"While psilocybin was successfully produced in yeast, the initial ti- ters were low. Furthermore, analysis revealed the extracellular accu- mulation of tryptamine (120.3 ± 11.1 mg/L) indicating a significant limitation in the conversion of tryptamine to 4-hydroxytryptamine encoded by the cytochrome P450 gene PcpsiH. Cytochrome P450 enzymes (CYP) belong to a superfamily of heme-containing mono- oxygenases and require a cytochrome P450 reductase (CPR) partner to deliver one or more electrons to reduce the heme-bound iron and oxi- dized substrates (Renault et al., 2014). While the detection of psilocybin indicated that the native S. cerevisiae CPR (Ncp1) could carry out this reduction, the accumulation of tryptamine suggested a sub-optimal"

"In this work, the successful production of psilocybin was demonstrated in Saccharomyces cerevisiae. Implementation of the biosynthetic pathway from Psilocybe cubensis with expression of a novel cytochrome P450 reductase resulted in high-level de novo production from glucose." ?